## Accuracy: 

things to change:
1. Kilometers/Charge Delta is blank for original script
2. str leg status is not matched
3. LastMod_SalesUser is not matched
4. TBL_RES_SEGMENTS join issue

things to note:
1. Route Pair doesn't matter?

In [1]:
from helpfile import *

# Import and clean snowflake views

In [2]:
query = """select * from ANALYTICS_PROD.IOATAWARE.VW_RESERVATION_DETAIL_RESERVATION_NUMBER where "Reservation Nmbr"=6297651
    """
snowflake_pd_df = get_data(query)

In [3]:
columns_to_remove = ['Charge Delta','Kilometers','str Leg Status','Last Mod Sales User','Route Pair']
snowflake_pd_df = snowflake_pd_df.drop(columns=columns_to_remove)

snowflake_pd_df['Charge Date'] = pd.to_datetime(snowflake_pd_df['Charge Date'])

snowflake_pd_df['Flight Date'] = pd.to_datetime(snowflake_pd_df['Flight Date'])
snowflake_pd_df['Flight Date'] =snowflake_pd_df['Flight Date'].dt.strftime('%m/%d/%Y')
snowflake_pd_df['Flight Date'] = pd.to_datetime(snowflake_pd_df['Flight Date'])

snowflake_pd_df['Net Charge'] = snowflake_pd_df['Net Charge'].astype(float)
snowflake_pd_df['Taxes'] = snowflake_pd_df['Taxes'].astype(float)
snowflake_pd_df['Total Charge'] = snowflake_pd_df['Total Charge'].astype(float)
snowflake_pd_df['Flight_Nmbr'] = snowflake_pd_df['Flight_Nmbr'].astype(int)
snowflake_pd_df['Base NetCharge'] = snowflake_pd_df['Base NetCharge'].astype(float)
snowflake_pd_df['Ancillary NetCharge'] = snowflake_pd_df['Ancillary NetCharge'].astype(float)
# snowflake_pd_df['Route Pair'] = snowflake_pd_df['Route Pair'].str.replace('  ', '')
# snowflake_pd_df['Route Pair'] = snowflake_pd_df['Route Pair'].str.replace(' ', '')
snowflake_pd_df['Description'] = snowflake_pd_df['Description'].str.replace(r'^\s+', '', regex=True)


snowflake_pd_df['Percent'] = snowflake_pd_df['Percent'].replace('%', '').astype(float)
snowflake_pd_df['Exchange Rate'] = snowflake_pd_df['Exchange Rate'].replace('%', '').astype(float)
snowflake_pd_df['Tax Percentage'] = snowflake_pd_df['Tax Percentage'].str.replace('%', '').astype(float)

snowflake_pd_df['Base Charge'] = snowflake_pd_df['Base Charge'].astype(float)
snowflake_pd_df['Base Discount'] = snowflake_pd_df['Base Discount'].astype(float)
snowflake_pd_df['Base Taxes'] = snowflake_pd_df['Base Taxes'].astype(float)

In [4]:
#snowflake_pd_df.dtypes

# Import and clean report data

In [5]:
report_df = pd.read_csv('Reservation_Detail/Reservation_Detail_Reservation_Number_6297651.csv', skiprows=0)   

In [6]:
columns_to_remove = ['str_Promo_Code','CASM','ChargeDelta','Kilometers','str_Leg_Status','LastMod_SalesUser','RoutePair']
report_df = report_df.drop(columns=columns_to_remove)

columns_to_process = ['NetCharge', 'Taxes','TotalCharge1','Base_NetCharge','Ancillary_NetCharge','Base_Charge','Base_Discount','Base_Taxes']

# Apply the function to specified columns in the dataframe
for col in columns_to_process:
    report_df[col] = report_df[col].apply(process_fee_column)

report_df['ChargeDate'] = pd.to_datetime(report_df['ChargeDate'])
report_df['ChargeDate'] =report_df['ChargeDate'].dt.strftime('%m/%d/%Y')
report_df['ChargeDate'] = pd.to_datetime(report_df['ChargeDate'])
report_df['FlightDate'] = pd.to_datetime(report_df['FlightDate'])
report_df['FlightDate'] =report_df['FlightDate'].dt.strftime('%m/%d/%Y')
report_df['FlightDate'] = pd.to_datetime(report_df['FlightDate'])
# report_df['RoutePair'] = report_df['RoutePair'].str.replace(' ', '')

report_df['Percent'] = report_df['Percent'].str.replace('%', '').astype(float)
report_df['mny_Exchange_Rate'] = report_df['mny_Exchange_Rate'].str.replace('%', '').astype(float)
report_df['mny_Tax_1_Percentage'] = report_df['mny_Tax_1_Percentage'].str.replace('%', '').astype(float)

report_df['mny_Distance'] = report_df['mny_Distance'].str.replace(',', '').astype(int)
report_df['Exchange_Rate'] = report_df['Exchange_Rate'].str.replace('%', '').astype(float)
report_df['Leg_TotalDistance'] = report_df['Leg_TotalDistance'].str.replace(',', '').astype(int)

C:\Users\Ruiying.Zhang\AppData\Local\Temp\ipykernel_30228\2628606820.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  report_df['FlightDate'] = pd.to_datetime(report_df['FlightDate'])


In [7]:
#report_df.dtypes

# Compare two dataframes

In [8]:
report_accuracy,snowflake_accuracy,report_only,snowflake_only = calculate_accuracy(report_df,snowflake_pd_df)
print(f"Accuracy: {snowflake_accuracy:.2f}%" " snowflake records in report\nThe number of records in snowflake but not in report is", len(snowflake_only))
print(f"Accuracy: {report_accuracy:.2f}%"" report records in snowflake\nThe number of records in report but not in snowflake is", len(report_only))

Accuracy: 78.57% snowflake records in report
The number of records in snowflake but not in report is 12
Accuracy: 73.33% report records in snowflake
The number of records in report but not in snowflake is 16


# Output the difference file

In [9]:
try:
    snowflake_only.to_csv("Reservation_Detail/"+"snowflake_only_record.csv", index=False) 
except:
    print("No snowflake only records")
try:
    report_only.to_csv("Reservation_Detail/"+"report_only_record.csv", index=False) 
except:
    print("No report only records")

In [10]:
snow_res = list(snowflake_pd_df['Reservation Nmbr'].unique())
report_res = list(report_df['Reservation Nmbr'].unique())

In [11]:
elements_in_snow_not_in_report = list(set(snow_res) - set(report_res))
elements_in_report_not_in_snow = list(set(report_res)-set(snow_res))

snow_file_path = "Sales/snowonly_reservation.txt"
report_file_path = "Sales/reportonly_reservation.txt"

# Open the file for writing
with open(snow_file_path, "w") as file:
    for item in elements_in_snow_not_in_report:
        file.write("%s," % item)
with open(report_file_path, "w") as file:
    for item in elements_in_snow_not_in_report:
        file.write("%s," % item)